%md
# Camada Gold – Análises e Tabelas Agregadas

## Documentação

**Objetivo:**  
Gerar tabelas analíticas agregadas com insights para tomada de decisão, a partir da camada Silver tratada e enriquecida.

**Entradas:**  
- Tabela Delta: `lakehouse.silver.yellow_trip`

**Saídas (Gold Tables):**  
- `lakehouse.gold.revenue_by_vendor`
- `lakehouse.gold.trips_by_payment`
- `lakehouse.gold.revenue_by_day`
- `lakehouse.gold.profitable_hour`
- `lakehouse.gold.monthly_revenue`

**Transformações aplicadas:**  
- Agrupamentos por data, empresa e tipo de pagamento
- Cálculo de faturamento total, média de distância, lucro por milha
- Extração de hora e dia de coleta (`pickup_datetime`)

## Decisões Tomadas

- Tabelas escritas com `mergeSchema` para permitir evolução de esquema.
- Insights escolhidos com base em perguntas de negócio comuns:
  - Quem mais fatura?
  - Qual a forma de pagamento mais usada?
  - Qual o dia mais lucrativo?
  - Em qual horário há maior rentabilidade?
- Automatização programada via **Databricks Workflows** para execução contínua.


In [0]:
import pyspark.sql.functions as F

In [0]:

#enriquecimento semântico dos dados da Silver
df_silver = spark.read.table("lakehouse.silver.yellow_trip") \
    .withColumn("vendor_name", F.expr("""
        CASE 
            WHEN vendor_id = 1 THEN 'Creative Mobile Technologies, LLC'
            WHEN vendor_id = 2 THEN 'Curb Mobility, LCC'
            WHEN vendor_id = 6 THEN 'Myle Technologies Inc'
            WHEN vendor_id = 7 THEN 'Helix'
            ELSE 'Desconhecido'
        END
    """)) \
    .withColumn("payment_description", F.expr("""
        CASE 
            WHEN payment_type = 0 THEN 'Flex Fare trip'
            WHEN payment_type = 1 THEN 'Credit card'
            WHEN payment_type = 2 THEN 'Cash'
            WHEN payment_type = 3 THEN 'No charge'
            WHEN payment_type = 4 THEN 'Dispute'
            WHEN payment_type = 5 THEN 'Unknown'
            WHEN payment_type = 6 THEN 'Voided trip'
            ELSE 'Outros'
        END
    """))


In [0]:
# Agrupar por fornecedor, ano e mês da corrida
df_monthly_revenue = df_silver.groupBy(
    "vendor_name",
    F.year("pickup_datetime").alias("year"),
    F.month("pickup_datetime").alias("month")
).agg(
    F.sum("total_amount").alias("total_revenue"),
    F.avg("trip_distance").alias("avg_distance"),
    F.count("pickup_datetime").alias("rides_per_day")
)

# Salvar os resultados como uma tabela na camada Gold com mesclagem de esquema habilitada
df_monthly_revenue.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.gold.monthly_revenue")

In [0]:
df_payment = df_silver.groupBy("vendor_name", "payment_description") \
    .count().orderBy("count")

df_payment.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.gold.rides_by_payment")

In [0]:
df_profitable_vendor = df_silver.groupBy("vendor_name") \
    .agg(F.round(F.sum("total_amount"),2).alias("total_revenue")) \
    .orderBy(F.desc("total_revenue"))

df_profitable_vendor.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.gold.revenue_by_vendor")

In [0]:
df_profitable_day = df_silver.withColumn("trip_date", F.to_date("pickup_datetime")) \
    .groupBy("vendor_name", "trip_date") \
    .agg(
        F.round(F.sum("total_amount"), 2).alias("total_revenue"),
        F.round(F.avg("trip_distance"), 2).alias("avg_distance")
    ) \
    .withColumn("revenue_per_mile", F.round( F.col("total_revenue") / F.col("avg_distance"),2)) \
    
df_profitable_day.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.gold.revenue_per_day")

In [0]:
# Perform the inner aggregation first
df_inner_agg = df_silver.withColumn("trip_date", F.to_date("pickup_datetime")) \
    .withColumn("trip_hour", F.hour("pickup_datetime")) \
    .groupBy("vendor_name", "trip_date", "trip_hour") \
    .agg(F.round(F.sum("total_amount"),2).alias("total_revenue"))

# Perform the outer aggregation
df_profitable_hour = df_inner_agg.groupBy("vendor_name", "trip_date", "trip_hour") \
    .agg(F.max("total_revenue").alias("total_revenue")) \
    .orderBy("trip_hour")

# Write the result to the table
df_profitable_hour.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("lakehouse.gold.profitable_hour")



In [0]:
tables = [t.name for t in spark.catalog.listTables("lakehouse.gold")]

for table in tables:
    df = spark.table(f"lakehouse.gold.{table}")
    df.write.mode("overwrite").option("header", "true").csv(f"s3://your-bucket-name/{table}.csv")